To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
import json
import pandas as pd
from datasets import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score
from trl import GRPOConfig, GRPOTrainer

In [4]:
from unsloth import is_bfloat16_supported
import torch
from vllm import SamplingParams
import re

In [5]:
# Load and preprocess your custom dataset
def load_custom_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def preprocess_custom_data(data):
    processed_data = []
    for item in data:
        # Ensure all elements in item["data"] are strings
        text = " ".join(map(str, item["data"]))
        labels = item["label"]
        processed_data.append({
            "prompt": text,  # Add prompt field
            "labels": labels
        })
    return Dataset.from_pandas(pd.DataFrame(processed_data))

In [6]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')  # 挂载到我的云盘的program下，需要验证，点继续即可

Mounted at /content/drive


In [7]:
# Load your dataset
train_data = load_custom_dataset("./数据/train_dataset.json")
val_data = load_custom_dataset("./数据/validation_dataset.json")

train_dataset = preprocess_custom_data(train_data)
val_dataset = preprocess_custom_data(val_data)


In [8]:
max_seq_length = 512  # Can increase for longer reasoning traces
lora_rank = 32  # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length=max_seq_length,
    load_in_4bit=True,  # False for LoRA 16bit
    fast_inference=True,  # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.6,  # Reduce if out of memory
)

INFO 03-05 13:59:54 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.4: Fast Llama patching. Transformers: 4.48.3. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 2.59 GB. Also swap space = 2 GB.
WARNING 03-05 14:00:01 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-05 14:00:16 config.py:5

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-05 14:02:01 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-05 14:02:01 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-05 14:02:15 worker.py:267] Memory profiling takes 13.17 seconds
INFO 03-05 14:02:15 worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.59) = 8.76GiB
INFO 03-05 14:02:15 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 0.74GiB; the rest of the memory reserved for KV Cache is 2.40GiB.
INFO 03-05 14:02:16 executor_base.py:111] # cuda blocks: 1226, # CPU blocks: 1024
INFO 03-05 14:02:16 executor_base.py:116] Maximum concurrency for 512 tokens per request: 38.31x
INFO 03-05 14:02:17 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occ

Capturing CUDA graph shapes: 100%|██████████| 23/23 [00:42<00:00,  1.85s/it]

INFO 03-05 14:03:00 model_runner.py:1562] Graph capturing finished in 43 secs, took 0.59 GiB
INFO 03-05 14:03:00 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 58.75 seconds


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],  # Remove QKVO if out of memory
    lora_alpha=lora_rank,
    use_gradient_checkpointing="unsloth",  # Enable long context finetuning
    random_state=3407,
)

Unsloth 2025.3.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [13]:
def ner_reward_func(prompts, completions, true_labels, **kwargs) -> list[float]:
    print("Completions:", completions)  # Debugging line
    responses = [completion[0]['content'] for completion in completions]
    predicted_labels = []
    for response in responses:
        # Assume model output format is "label1 label2 label3 ..."
        predicted_labels.append([int(label) for label in response.split()])

    # Calculate F1 score
    f1_scores = []
    for true, pred in zip(true_labels, predicted_labels):
        f1 = f1_score(true, pred, average="weighted")
        f1_scores.append(f1)
    return f1_scores

In [14]:
# Training configuration
training_args = GRPOConfig(
    use_vllm=True,  # use vLLM for fast inference!
    learning_rate=5e-6,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    logging_steps=1,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,  # Increase to 4 for smoother training
    num_generations=6,  # Decrease if out of memory
    max_prompt_length=256,
    max_completion_length=200,
    max_steps=250,
    save_steps=250,
    max_grad_norm=0.1,
    report_to="none",  # Can use Weights & Biases
    output_dir="outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


In [15]:
# Initialize trainer
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        lambda prompts, completions, **kwargs: ner_reward_func(prompts, completions, true_labels=train_dataset["labels"]),
    ],
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 452 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/4,712,566,784 (1.78% trained)


Completions: ["<|reserved_special_token_149|>I agree to the terms of the survey, but please clarify that the question is an important one and I will provide an honest answer. I will maintain confidentiality and respect the privacy of the survey. What questions should I answer? What is the survey about? | ||\nThe provided text is a response rather than a passage that can be summarized. However, I'll attempt to provide a clear and concise response to emulate a summary, assuming the original passage is a survey introduction or an explanation of the survey's purpose.\n\nGiven the seemingly unrelated sentences, it appears there might have been an error in the passage's construction or interpretation. Taking the apparent introduction at face value, here is a possible clarification and summary:\n\nThe passage appears to be a respondent's declaration of agreement to terms before answering a survey. It mentions agreeing to the terms, being honest in their answer, and respecting the privacy of t

TypeError: string indices must be integers, not 'str'

In [ ]:
# Evaluate the model
def evaluate_model(dataset):
    total_f1 = 0.0
    for example in dataset:
        text = example["text"]
        true_labels = example["labels"]
        prompt = tokenizer.apply_chat_template([
            {"role": "user", "content": text},
        ], tokenize=False, add_generation_prompt=True)
        sampling_params = SamplingParams(
            temperature=0.8,
            top_p=0.95,
            max_tokens=1024,
        )
        output = model.fast_generate(
            [prompt],
            sampling_params=sampling_params,
            lora_request=None,
        )[0].outputs[0].text
        predicted_labels = [int(label) for label in output.split()]
        f1 = f1_score(true_labels, predicted_labels, average="weighted")
        total_f1 += f1
    return total_f1 / len(dataset)

In [ ]:
val_f1 = evaluate_model(val_dataset)
print(f"Validation F1 Score: {val_f1}")

# Save the model
model.save_lora("ner_saved_lora")